In [1]:
from nltk.tokenize import TweetTokenizer
from nltk.stem.snowball import RussianStemmer
import pandas as pd
import re
import tensorflow as tf
import numpy as np
import sys
import vk
import telebot


In [2]:
t = TweetTokenizer()
stemer = RussianStemmer()
regex = re.compile('[^а-яА-Я ]')

In [3]:
length_of_the_dictionary = 6 #stem words
tweets_col_number = 3
negative_tweets = pd.read_csv(
    'negative.csv', header=None, delimiter=';')[[tweets_col_number]]
positive_tweets = pd.read_csv(
    'positive.csv', header=None, delimiter=';')[[tweets_col_number]]

In [4]:
sess = tf.Session()
w = tf.get_variable("Weights", [310000,1], dtype=tf.float64, initializer=tf.glorot_uniform_initializer(
    seed=None,
    dtype=tf.float64
))
sess.run (tf.global_variables_initializer())
dictionary = []
expected = tf.placeholder(tf.float64)
x = tf.placeholder(tf.float64)
predict = tf.sigmoid(tf.matmul(x,w))
loss = tf.pow(predict - expected,2)
Optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

In [8]:
for i,tweet in positive_tweets.iterrows():
    for token in t.tokenize(tweet[3]):
        vector = np.zeros((1,310000))
        stem = stemer.stem(token)
        if stem not in dictionary: dictionary.append(stem)
        vector[0,dictionary.index(stem)] = 1.0
        sess.run([loss,Optimizer],feed_dict={x: vector,expected: 1.0})
    
    
    if len(dictionary)>=400:break

print ('Positive is done !')


for i,tweet in negative_tweets.iterrows():
    for token in t.tokenize(tweet[3]):
        vector = np.zeros((1,310000))
        stem = stemer.stem(token)
        if stem not in dictionary: dictionary.append(stem)
        vector[0,dictionary.index(stem)] = 1.0
        sess.run([loss,Optimizer],feed_dict={x: vector,expected: 0.0})
    if len(dictionary)>=800:break     
    
print ('Negative is done !')
    


Positive is done !
Negative is done !


In [9]:
vector = np.zeros((1,310000))   
for i in t.tokenize('Какой то текст'):
    
    
    print (stemer.stem(i))
    print (dictionary.index(stemer.stem(i)))
    vector[0,dictionary.index(stemer.stem(i))] = 1.0
print (sess.run(predict,feed_dict={x: vector}))

[[ 0.1781732]]


In [ ]:
bot = telebot.TeleBot('519802622:AAERN18WJLVog4eCLhl6lnB2htkJM8urV-4')
@bot.message_handler(content_types=["text"])
def repeat_all_messages(messag):
    vector = np.zeros((1,310000))   
    for i in t.tokenize(messag.text):
        
        vector[0,dictionary.index(stemer.stem(i))] = 1.0
        
    bot.send_message(messag.chat.id,sess.run(predict,feed_dict={x: vector}) )    
   
    




In [ ]:
bot.polling(none_stop=True)

In [ ]:
dictionary[4]

In [10]:

sess.run (w[117,0])

0.17302774741298055

In [11]:
sess.run (w[2,0])

-1.9484499896774039

In [4]:
negative_tweets

,3
0,на работе был полный пиддес :| и так каждое за...
1,"Коллеги сидят рубятся в Urban terror, а я из-з..."
2,@elina_4post как говорят обещаного три года жд...
3,"Желаю хорошего полёта и удачной посадки,я буду..."
4,"Обновил за каким-то лешим surf, теперь не рабо..."
5,"Котёнка вчера носик разбила, плакала и расстра..."
6,"@juliamayko @O_nika55 @and_Possum Зашли, а то ..."
7,а вообще я не болею - я не выздоравливаю :(
8,я микрофраза :( учимся срать кирпичами в режим...
9,"я хочу с тобой помириться , но сука я гордая и..."


In [ ]:
tf.nn.sparse_softmax_cross_entropy_with_logits